In [22]:
%matplotlib inline

import json
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
import gensim.downloader as api

info = api.info()  # show info about available models/datasets
model = api.load("glove-twitter-25")  # download the model and return as object ready for use

nlp = spacy.load('en')

[==================================================] 100.0% 104.8/104.8MB downloaded


Read the Readme here:
https://github.com/RaRe-Technologies/gensim-data

# Getting the dataset

We will use here the dataset provided by the authors we will create a helper function to extract the features that we want.

In [23]:
data = pd.read_json("diplomacy_data/diplomacy_data.json")

In [24]:
def last_support(entry):
    """
    This function returns the last season of friendship. The code is inspired by the provided code from
    the authors
    """
    last_support = None
    for season in entry[:-1]:
        if 'support' in season['interaction'].values():
            last_support = season['season']
    return last_support

def treat_msg_season(df):
    """
    This function loops over the whole dataset and creates a dictionnary with the set of features for each season with 
    its associated boolean (betrayal or not )
    """
    tab_vi = []
    data_victim = {'features':[], 'betrayed':[]} # data of the (potential) victim 
    data_betrayer = {'features':[], 'betrayed':[]} # data of the (potential) betrayer
    for i in range(len(df.seasons.values)):
        entry = df['seasons'][i] # pick each entry
        for j in range(len(entry)): # pick each season
            season = entry[j]
            tab_vi = []
            tab_be = []
            if season['season'] <= last_support(entry): # check if the season is below the last season of friendship
                tab_vi.append(season['messages']['victim'])
                tab_be.append(season['messages']['betrayer'])
                if len(tab_be) != 0 and len(tab_vi) != 0: # keep only cases where both players have sent messages
                    data_victim['features'].append(tab_vi)
                    data_victim['betrayed'].append(df.betrayal.values[i])
                    data_betrayer['features'].append(tab_be)   
                    data_betrayer['betrayed'].append(df.betrayal.values[i])
    return data_victim, data_betrayer

data_victim, data_betrayer = treat_msg_season(data)

In [25]:
def get_nb_msg(data):
    """
    Get the mean number of messages sent per season
    """
    tab = []
    for features in data["features"]:
        tab.append(len(features[0]))
    return tab

print("In each season, potential betrayers send in average {}, with a maximum of {} messages".format(np.mean(get_nb_msg(data_betrayer)), np.max(get_nb_msg(data_betrayer))))
print("In each season, potential victims send in average {}, with a maximum of {} messages".format(np.mean(get_nb_msg(data_victim)), np.max(get_nb_msg(data_victim))))

In each season, potential betrayers send in average 1.627498001598721, with a maximum of 38 messages
In each season, potential victims send in average 1.515587529976019, with a maximum of 28 messages


In [26]:
def get_lexicon_words(entry):
    """
    get the set of lexicon words for each entry of the dataset
    1 entry = 1 row = 1 set of messages
    Can be improved
    """
    for entries in entry[0]: #loop over the messages
        # get the lexicon words
        di_words = entries["lexicon_words"]
        tab_words = []
        for key in di_words:
            tab = di_words[key]
            for words in tab:
                word = words.split(' ')
                for w in word:
                    if w not in tab_words:
                        tab_words.append(w)
    return tab_words

test = get_lexicon_words(data_victim["features"][-1])

# NLP pipeline

To implement : remove stop words

# Spacy & Glove word embedding

Here we use the spacy library to compute the embeddings. We have to try also with Glove and Word2Vec

In [40]:
from scipy.spatial import distance

def get_word_embedding(word):
    return nlp(word).vector

def get_word_embedding_model(model, word):
    return model[word]

print("Word :{} , embedding : {}".format(test[1], get_word_embedding_model(model, test[1])))
#model.most_similar("man")

Word :still , embedding : [-0.22114   0.67529   0.59344  -1.0235   -0.6578    0.64357   1.6461
 -0.13754  -0.20652   0.41388  -0.12224   0.92581  -5.0168   -0.11061
  0.034176  0.35356   0.027989 -0.55968  -0.2286   -0.79967   0.58868
  0.56942   0.29349   0.3104   -0.50146 ]


# Word2Vec embedding